## NDN Content Revoking through Key Cycling

Larry Huang (lhuan130@asu.edu)

This section of the project includes my attempts at producing a working NDN-DPDK slice.
Non-functional. If you can get NDN-DPDK to properly install by fixing the scripts below, you have surpassed my willingness to fight build issues.

Produced on the Summer 2024 version of FABRIC's JuPyter hub. If you have not yet configured your FABRIC environment, return to the start_here notebook before starting this.

### Deallocate Slice

This content is placed at the top to avoid mistakes from people running through the notebook using Shift-Enter.
Note that `fablib` is not imported unless the first code block after this section is run. 

In [ ]:
slice_name="NDNkeyCycle-" + fablib.get_bastion_username()
slice = fablib.get_slice(name=slice_name)
fablib.delete_slice(slice_name)

### Initial Setup (from fabric-ndn)

The bastion key and slice key must be in your fabric_config JuPyter directory in order to interact with the FabLib library on FABRIC.

Try to avoid skipping steps to prevent missing variables/imports.

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()
conf = fablib.show_config()

### Topology Configuration (run to load variables)

In [ ]:
slice_name="NDNkeyCycle-" + fablib.get_bastion_username()

#at current the topology is intended to allocate on a single site
#it may be more prudent to allocate across multiple sites as more nodes are added due to the large size requirements

#configuring nodes for topology in *_conf variables
#Large ram and disk values are a result of NDN-DPDK's space reqs
node_conf = [
    {"name": "attacker", "cores": 2, "ram": 72, "disk": 60, "image": 'default_ubuntu_20', 'packages': ['net-tools', 'iperf3', 'moreutils']},
    {"name": "consumer", "cores": 2, "ram": 72, "disk": 60, "image": 'default_ubuntu_20', 'packages': ['net-tools', 'iperf3', 'moreutils']},
    {"name": "producer", "cores": 2, "ram": 72, "disk": 60, "image": 'default_ubuntu_20', 'packages': ['net-tools', 'iperf3', 'moreutils']},
    {"name": "router1",  "cores": 2, "ram": 72, "disk": 60, "image": 'default_ubuntu_20', 'packages': ['net-tools', 'iperf3', 'moreutils']}
]

net_conf = [
    {"name": "net_clients", "subnet": "10.10.1.0/24", "nodes": [
        {"name": "consumer", "addr": "10.10.1.30"},
        {"name": "attacker", "addr": "10.10.1.40"},
        {"name": "router1",  "addr": "10.10.1.10"}
    ]},
    {"name": "net_content", "subnet": "10.10.2.0/24", "nodes": [
        {"name": "producer",  "addr": "10.10.2.50"},
        {"name": "router1",   "addr": "10.10.2.10"}
    ]}
]

route_conf = [
    {"addr": "10.10.1.0/24", "gw": "10.10.1.10", "nodes": ["consumer", "attacker"]},
    {"addr": "10.10.2.0/24", "gw": "10.10.2.10", "nodes": ["producer"]}
]

#calculate config values for use in determining a valid site for allocation
exp_conf = {'cores': sum([ n['cores'] for n in node_conf]), 'nic': sum([len(n['nodes']) for n in net_conf]), 'ram': sum([ n['ram'] for n in node_conf]) }

print("Configuration setup loaded.")

### Slice existence check and allocation

In [ ]:
slice_name="NDNkeyCycle-" + fablib.get_bastion_username()

try:
    slice = fablib.get_slice(name=slice_name)
    print("You already have a slice by this name!")
    print("If you previously reserved resources, skip to the 'log in to resources' section.")
except:
    print("You don't have a slice named %s yet." % slice_name)
    print("Continue to the next step to make one.")
    slice = fablib.new_slice(name=slice_name)

In [ ]:
## Run if Slice is not allocated

while True:
    site_name = fablib.get_random_site()
    #check if randomly chosen site has the resources to properly allocate
    if ( (fablib.resources.get_core_available(site_name) > 1.2*exp_conf['cores']) and
        (fablib.resources.get_ram_available(site_name) > 1.2*exp_conf['ram']) and
        (fablib.resources.get_component_available(site_name, 'SharedNIC-ConnectX-6') > 1.2**exp_conf['nic']) ):
        break

for n in node_conf:
    slice.add_node(name=n['name'], site=site_name, 
                   cores=n['cores'], 
                   ram=n['ram'], 
                   disk=n['disk'], 
                   image=n['image'])
for n in net_conf:
    ifaces = [slice.get_node(node["name"]).add_component(model="NIC_Basic", 
                                                 name=n["name"]).get_interfaces()[0] for node in n['nodes'] ]
    slice.add_l2network(name=n["name"], type='L2Bridge', interfaces=ifaces)

fablib.show_site(site_name)

In [ ]:
#wait for visible output from previous
#it may be helpful to spot-check the resources availble on the other node
slice.submit()

#wait for allocation success
slice.get_state()
slice.wait_ssh(progress=True)

#NOTE: examine output of this and the above block carefully to ensure the slice has been properly built 

### Configure resources within nodes

Step 1: Update Nodes and Install Useful Packages

In [ ]:
for n in node_conf:
    if len(n['packages']):
        node = slice.get_node(n['name'])
        pkg = " ".join(n['packages'])
        node.execute_thread("sudo apt update; sudo apt -y install %s" % pkg)

from ipaddress import ip_address, IPv4Address, IPv4Network
for net in net_conf:
    for n in net['nodes']:
        if_name = n['name'] + '-' + net['name'] + '-p1'
        iface = slice.get_interface(if_name)
        iface.ip_link_up()
        if n['addr']:
            iface.ip_addr_add(addr=n['addr'], subnet=IPv4Network(net['subnet']))
        else:
            iface.get_node().execute("sudo ip addr flush dev %s"  % iface.get_device_name())

hosts_txt = [ "%s\t%s" % ( n['addr'], n['name'] ) for net in net_conf  for n in net['nodes'] if type(n) is dict and n['addr']]

for n in slice.get_nodes():
    for h in hosts_txt:
        n.execute("echo %s | sudo tee -a /etc/hosts" % h)

for n in slice.get_nodes():
    n.execute("sudo sysctl -w net.ipv4.ip_forward=1")

for rt in route_conf:
    for n in rt['nodes']:
        slice.get_node(name=n).ip_route_add(subnet=IPv4Network(rt['addr']), gateway=rt['gw'])

In [ ]:
#If you want to perform the next steps manually, this will be useful

for node in slice.get_nodes():
    print(node.get_name())
    print(node.get_ssh_command())

#### NDN Installation and Node Environment Configuration

Step 2: Install NDN-DPDK (taken from FABRIC-NAC)

Included in first code block below. Or load the following script into each node and run.

```
#!/bin/bash
sudo apt update
wget http://www.mellanox.com/downloads/ofed/MLNX_OFED-5.8-1.0.1.1/MLNX_OFED_SRC-debian-5.8-1.0.1.1.tgz
tar zxvf MLNX_OFED_SRC-debian-5.8-1.0.1.1.tgz
sudo MLNX_OFED_SRC-5.8-1.0.1.1/./install.pl
git clone https://github.com/usnistgov/ndn-dpdk
git clone https://github.com/DPDK/dpdk
sudo apt install --no-install-recommends -y ca-certificates curl jq lsb-release sudo nodejs
chmod a+x /home/ubuntu/ndn-dpdk/docs/ndndpdk-depends.sh
echo | /home/ubuntu/ndn-dpdk/docs/./ndndpdk-depends.sh
sudo npm install -g pnpm
cd /home/ubuntu/ndn-dpdk/core && pnpm install
cd /home/ubuntu/ndn-dpdk && NDNDPDK_MK_RELEASE=1 make && sudo make install
sudo python3 /home/ubuntu/dpdk/usertools/dpdk-hugepages.py -p 1G --setup 64G
sudo ndndpdk-ctrl systemd start
ndndpdk-ctrl -v
```

Please note: an absolutely HUGE amount of text will be produced below this block, and it may take anywhere from ten minutes to two hours to finish running. Please be patient with this section.

NOTE: current problem points include the following:
- Output from these blocks will print excessively below this block. If you wish to view the output of individual nodes, the module below will actually save output from each attempt into individual txt files in the JuPyter directory for review.
- Trying to install npm runs into issues with recommendation blocks where installing it without some recommended items causes it to break when used subsequently.
- Subsequently, trying to use npm to install pnpm creates issues where FABRIC nodes are not allowed to reach out to appropriate download targets. pnpm is required to build and install NDN-DPDK.

Things that are not actually problems:
- if you have already run the above script once and it has failed, you will receive errors when trying to clone dpdk and ndn-dpdk again. This is expected and will not block progress.

In [ ]:
#Installing NDN-DPDK based upon FABRIC-NAC setup

from fabrictestbed_extensions.fablib.fablib import fablib
import traceback
import datetime
import threading
import json

def NDN_install(name: str):
    commands = [
        f"echo \"PS1=\'{name}:\\w\\$ \'\" >> .bashrc",
        "sudo apt update",
        "sudo apt install -y npm --no-install-recommends --no-install-suggests",
        "wget http://www.mellanox.com/downloads/ofed/MLNX_OFED-5.8-1.0.1.1/MLNX_OFED_SRC-debian-5.8-1.0.1.1.tgz",
        "tar zxvf MLNX_OFED_SRC-debian-5.8-1.0.1.1.tgz",
        "sudo MLNX_OFED_SRC-5.8-1.0.1.1/./install.pl",
        "git clone https://github.com/usnistgov/ndn-dpdk",
        "git clone https://github.com/DPDK/dpdk",
        "sudo apt install --no-install-recommends -y ca-certificates curl jq lsb-release sudo nodejs",
        "chmod a+x ndn-dpdk/docs/ndndpdk-depends.sh",
        "echo | ndn-dpdk/docs/./ndndpdk-depends.sh",
        "sudo npm install -g pnpm",
        "cd ndn-dpdk/core && pnpm install",
        "cd ndn-dpdk && NDNDPDK_MK_RELEASE=1 make && sudo make install",
        "sudo python3 dpdk/usertools/dpdk-hugepages.py -p 1G --setup 64G",
        "sudo ndndpdk-ctrl systemd start",
        "ndndpdk-ctrl -v"
    ]
    node = slice.get_node(name=name)
    console_data = []
    try:
        #if you wish to see all output, use stdout instead of consoleout
        consoleout, stderr = node.execute("ndndpdk-ctrl -v")
        if consoleout.startswith("ndndpdk-ctrl version"):
            print(f"Already installed on {name}")
            return
        console_data.append(consoleout)
        console_data.append('\n')
        for command in commands:
            print(f"Executing {command} on {name}")
            consoleout, stderr = node.execute(command)
            console_data.append(consoleout)
        if consoleout.startswith("ndndpdk-ctrl version"):
            print(f"Success on {name} at {datetime.datetime.now()}")
        else:
            print(f"Failure on {name} at {datetime.datetime.now()}")
    except Exception:
        print(f"Failed: {name} at {datetime.datetime.now()}")
    finally:
        write_targ = open(str(int(datetime.datetime.timestamp(datetime.datetime.now())))+"node_"+name+"NDNsetup.txt", 'w')
        write_targ.writelines(console_data)
        write_targ.close()

print(f"Starting: {datetime.datetime.now()}")
for node in slice.get_nodes():
    threading.Thread(target=NDN_install, args=(node.get_name(),)).start()

### Node Environment Setup (used for specifics of NDN exchange demo)
Step 3: set up file systems for the NDN nodes.
Step 4: install pwntools on the nodes to enable interactions with NDN-DPDK using the scripts modified from FABRIC_NAC.


In [ ]:
def build_user(node):
    print("Adding client environment for NDN node " + node.get_name())
    try:
        #partial NAC scripts sourced from FABRIC NAC
        result = node.upload_file('oldScripts/consumer.py','oldScripts/consumer.py')
        result = node.upload_file('oldScripts/producer.py','oldScripts/producer.py')

nodes = slice.get_nodes()
try:
    for node in nodes:
        build_user(node)
except Exception as e:
    print(f"Failed: {name} with {e.message}")

### (Incomplete) Procedure to demonstrate full NAC using NDN-DPDK

### Extend Slice Lifetime by 3 Days

In [ ]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

# Set end date to 3 days from now
end_date = (datetime.now(timezone.utc) + timedelta(days=4)).strftime("%Y-%m-%d %H:%M:%S %z")
slice.renew(end_date)

### Generate Node Access Details Below

In [ ]:
#Display access to nodes through ssh
for node in slice.get_nodes():
    print(node.get_name())
    print(node.get_ssh_command())

In [ ]:
#Draw topology for visibility
l2_nets = [(n.get_name(), {'color': 'lavender'}) for n in slice.get_l2networks() ]
l3_nets = [(n.get_name(), {'color': 'pink'}) for n in slice.get_l3networks() ]
hosts   =   [(n.get_name(), {'color': 'lightblue'}) for n in slice.get_nodes()]
nodes = l2_nets + l3_nets + hosts
ifaces = [iface.toDict() for iface in slice.get_interfaces()]
edges = [(iface['network'], iface['node'], 
          {'label': iface['physical_dev'] + '\n' + iface['ip_addr'] + '\n' + iface['mac']}) for iface in ifaces]
import networkx as nx
import matplotlib.pyplot as plt
plt.figure(figsize=(len(nodes) + 5, len(nodes) + 1))
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)
pos = nx.spring_layout(G)
nx.draw(G, pos, node_shape='s',  
        node_color=[n[1]['color'] for n in nodes], 
        node_size=[len(n[0])*400 for n in nodes],  
        with_labels=True);
nx.draw_networkx_edge_labels(G,pos,
                             edge_labels=nx.get_edge_attributes(G,'label'),
                             font_color='gray', font_size=8, rotate=False);